In [6]:
import numpy as np
from tqdm import tqdm
import pickle
import sqlite3
import pandas as pd
from gensim.models import Word2Vec
import polars as pl
import difflib

In [14]:
model = Word2Vec.load(r"C:\Users\Sam\Spotify-Million-Playlist-Challenge\word2vec.model")

In [2]:
conn = sqlite3.connect(r'C:\Users\Sam\spotify\spotify.db')

In [17]:
query = f"""
    SELECT playlist.pid, playlist.playlist_name
    FROM playlist
    LIMIT 100000
    """
playlist_df = pl.read_database(query, conn)

In [4]:
playlist_df

pid,playlist_name
i64,str
0,"""Throwbacks"""
1,"""Awesome Playlist"""
2,"""korean """
3,"""mat"""
4,"""90s"""
…,…
995,"""old"""
996,"""Daze"""
997,"""rap"""


In [26]:
 playlist_df.filter(pl.col("playlist_name") == "nicki minaj")["pid"].to_list()

[81882]

In [45]:
from typing import List


def get_closest_pids(playlist_name: str, n=3) -> List[int]:
    closest_playlist_names = difflib.get_close_matches(
        playlist_name, playlist_df["playlist_name"].to_list(), 
        n=n, cutoff=0
    )
    print(f"Matched {playlist_name} to {closest_playlist_names}")
    return playlist_df.filter(pl.col("playlist_name").is_in(closest_playlist_names))["pid"].to_list()

In [46]:
pids = get_closest_pids("nicki minaj")

Matched nicki minaj to ['nicki minaj', 'Nicki Minaj', 'Nicki Minaj']


In [47]:
pids_str = ""
for pid in pids:
    pids_str += f"{pid}, "
pids_str = pids_str[:-2]
pids_str

'11877, 23406, 37946, 43337, 46950, 54851, 68126, 71623, 81882, 97805'

In [48]:
query = f"""
    SELECT PT.pid, GROUP_CONCAT(PT.track_uri)
    FROM playlist_track PT
    WHERE PT.pid IN ({pids_str})
    GROUP BY PT.pid;
    """
playlist_tracks = pl.read_database(query, conn)

In [58]:
playlist_tracks["GROUP_CONCAT(PT.track_uri)"].str.len_bytes().max()

4180

In [63]:
conn = sqlite3.connect(r'C:\Users\Sam\spotify\spotify.db')

query = f"""
    SELECT playlist.pid, playlist.playlist_name
    FROM playlist
    LIMIT 100000
    """
playlist_df = pl.read_database(query, conn)

def get_closest_pids(playlist_name: str, n=3) -> List[int]:
    closest_playlist_names = difflib.get_close_matches(
        playlist_name, playlist_df["playlist_name"].to_list(), 
        n=n, cutoff=0
    )
    print(f"Matched {playlist_name} to {closest_playlist_names}")
    return playlist_df.filter(pl.col("playlist_name").is_in(closest_playlist_names))["pid"].to_list()

def get_tracks_of_similar_playlist(playlist_name: str) -> List[str]:
    pids = get_closest_pids(playlist_name)
    pids_str = ""
    for pid in pids:
        pids_str += f"{pid}, "
    pids_str = pids_str[:-2]
    query = f"""
        SELECT PT.pid, GROUP_CONCAT(PT.track_uri)
        FROM playlist_track PT
        WHERE PT.pid IN ({pids_str})
        GROUP BY PT.pid;
        """
    playlist_tracks = pl.read_database(query, conn)
    return playlist_tracks.filter(pl.col("GROUP_CONCAT(PT.track_uri)").str.len_bytes() == playlist_tracks["GROUP_CONCAT(PT.track_uri)"].str.len_bytes().max())["GROUP_CONCAT(PT.track_uri)"].item().split(',')

get_tracks_of_similar_playlist("nicki minaj")

Matched nicki minaj to ['nicki minaj', 'Nicki Minaj', 'Nicki Minaj']


['spotify:track:1yQunV0upd4Eyu3rcOerD9',
 'spotify:track:5h5sJvkKdIfC4yjALOmPn7',
 'spotify:track:2Gb3ADUtL6eCribC2T7gqX',
 'spotify:track:6bzwoiUt0s1KDOedyy4OtQ',
 'spotify:track:1CSQgn4dTVadTozLlAqbrV',
 'spotify:track:3y9cCbnBn0zjkJa2YkOj70',
 'spotify:track:0d6gYlrea7zwNsjfAh9Xba',
 'spotify:track:63NTlJWNS65JjsE3QJYr20',
 'spotify:track:5x8W6iNXDCQrHp1DF5j4UW',
 'spotify:track:7KcGEssn7BnJdTgildK5y0',
 'spotify:track:7gKIt3rDGIMJDFVSPBnGmj',
 'spotify:track:6X7xF5He9mB9vFsu56NAe1',
 'spotify:track:0Jm9XTLs2VP52VvzGWiEAH',
 'spotify:track:5JuE6xmUGOm3a2o7U18uTU',
 'spotify:track:2iA8RRYseF4Nif8UxWfNXR',
 'spotify:track:39XNWKuuBZK0oaV5YmD2dF',
 'spotify:track:4dL6PO16Ns1kmvzpj9lTFq',
 'spotify:track:6mjEABpi5cQ5gqFFOkR1Cc',
 'spotify:track:7oZzcRnS5j3enhjGUJTrBW',
 'spotify:track:1uqLxPxVgM50Dzoe6l3L00',
 'spotify:track:2RV8KsoOJbKdTURDpRnP6I',
 'spotify:track:5moUyr5BliVwQpLYzfn5tr',
 'spotify:track:13VDHbpoKlRhtgUaWTE2Ld',
 'spotify:track:4VRAVd6drMjJO4AvRLrd7w',
 'spotify:track:

In [62]:
playlist_tracks.filter(pl.col("GROUP_CONCAT(PT.track_uri)").str.len_bytes() == playlist_tracks["GROUP_CONCAT(PT.track_uri)"].str.len_bytes().max())["GROUP_CONCAT(PT.track_uri)"].item().split(',')

['spotify:track:1yQunV0upd4Eyu3rcOerD9',
 'spotify:track:5h5sJvkKdIfC4yjALOmPn7',
 'spotify:track:2Gb3ADUtL6eCribC2T7gqX',
 'spotify:track:6bzwoiUt0s1KDOedyy4OtQ',
 'spotify:track:1CSQgn4dTVadTozLlAqbrV',
 'spotify:track:3y9cCbnBn0zjkJa2YkOj70',
 'spotify:track:0d6gYlrea7zwNsjfAh9Xba',
 'spotify:track:63NTlJWNS65JjsE3QJYr20',
 'spotify:track:5x8W6iNXDCQrHp1DF5j4UW',
 'spotify:track:7KcGEssn7BnJdTgildK5y0',
 'spotify:track:7gKIt3rDGIMJDFVSPBnGmj',
 'spotify:track:6X7xF5He9mB9vFsu56NAe1',
 'spotify:track:0Jm9XTLs2VP52VvzGWiEAH',
 'spotify:track:5JuE6xmUGOm3a2o7U18uTU',
 'spotify:track:2iA8RRYseF4Nif8UxWfNXR',
 'spotify:track:39XNWKuuBZK0oaV5YmD2dF',
 'spotify:track:4dL6PO16Ns1kmvzpj9lTFq',
 'spotify:track:6mjEABpi5cQ5gqFFOkR1Cc',
 'spotify:track:7oZzcRnS5j3enhjGUJTrBW',
 'spotify:track:1uqLxPxVgM50Dzoe6l3L00',
 'spotify:track:2RV8KsoOJbKdTURDpRnP6I',
 'spotify:track:5moUyr5BliVwQpLYzfn5tr',
 'spotify:track:13VDHbpoKlRhtgUaWTE2Ld',
 'spotify:track:4VRAVd6drMjJO4AvRLrd7w',
 'spotify:track:

In [31]:
query = f"""
    SELECT track.track_uri, track.track_name, artist.artist_name
    FROM track
    JOIN album on track.album_uri = album.album_uri
    JOIN artist ON album.artist_uri = artist.artist_uri
    """
track_df = pl.read_database(query, conn)

In [42]:
track_df.select(pl.col("track_name"))

track_name
str
"""Sinnerman - Live In New York/1…"
"""Feeling Good"""
"""I've Got A Woman"""
"""Fly Me To The Moon"""
"""That's Life"""
…
"""Piano Sonata No.8 In C Minor, …"
"""French Suite No. 4 in E-flat M…"
"""Bach Concerto in D Minor for T…"


In [49]:
track_df["track_name"].str.contains("Love Story").any()

True

In [62]:
track_df.filter((pl.col("artist_name") == "50 Cent"))

track_uri,track_name,artist_name
str,str,str
"""spotify:track:7iL6o9tox1zgHpKU…","""In Da Club""","""50 Cent"""
"""spotify:track:3VzJE6yGuj8fDExU…","""Candy Shop""","""50 Cent"""
"""spotify:track:2D1hlMwWWXpkc3CZ…","""P.I.M.P.""","""50 Cent"""
"""spotify:track:41bIQPBE1lFN0mmw…","""21 Questions""","""50 Cent"""
"""spotify:track:2a4FxeAvHco9UkaV…","""Too Rich For The Bitch""","""50 Cent"""
…,…,…
"""spotify:track:1yF4OLqi6CeSeYk2…","""Intro/ 50 Cent/ The Massacre""","""50 Cent"""
"""spotify:track:3DDYWGvWmTPfvrXZ…","""Baby By Me - Benny Benassi Rem…","""50 Cent"""
"""spotify:track:4mrgISZAsHSrHde2…","""In Da Club - A Cappella""","""50 Cent"""


In [58]:
def get_closest(artist: str, track: str, n: int):
    if not track_df["track_name"].str.contains(track).any():
        print("track not found in database")
    if not track_df["artist_name"].str.contains(artist).any():
        print("artist not found in database")
    print("Finding track uri")
    track_uri = track_df.filter((pl.col("artist_name") == artist) & (pl.col("track_name") == track)).select(pl.col("track_uri"))[0].item()
    print(f"Track URI: {track_uri}")
    print("The most similar songs are:")
    for track_uri, similarity in model.wv.most_similar(track_uri, topn=n):
        query = f"""
            SELECT track.track_name, artist.artist_name
            FROM track
            JOIN album on track.album_uri = album.album_uri
            JOIN artist ON album.artist_uri = artist.artist_uri
            WHERE track.track_uri == '{track_uri}';
            """
        single_track_df = pl.read_database(query, conn)
        print(f" > {similarity:.3f} - {single_track_df['artist_name'][0]}: {single_track_df['track_name'][0]}")

In [64]:
get_closest("50 Cent", "Many Men (Wish Death)", 50)

track not found in database
Finding track uri
Track URI: spotify:track:2I9foKseoFQh07p6sD2voE
The most similar songs are:
 > 0.890 - 50 Cent: I'll Whip Ya Head Boy
 > 0.873 - 50 Cent: Back Down
 > 0.872 - 50 Cent: I'm Supposed To Die Tonight
 > 0.872 - 50 Cent: U Not Like Me
 > 0.869 - 50 Cent: Window Shopper
 > 0.867 - 50 Cent: My Gun Go Off
 > 0.866 - 50 Cent: What Up Gangsta
 > 0.866 - 50 Cent: Blood Hound
 > 0.864 - 50 Cent: Hustler's Ambition
 > 0.859 - 50 Cent: Like My Style
 > 0.858 - 50 Cent: Ski Mask Way
 > 0.857 - 50 Cent: So Disrespectful
 > 0.855 - 50 Cent: Patiently Waiting
 > 0.854 - 50 Cent: In My Hood
 > 0.853 - 50 Cent: A Baltimore Love Thing
 > 0.849 - 50 Cent: When It Rains It Pours
 > 0.849 - 50 Cent: Lifes On The Line
 > 0.849 - 50 Cent: Gotta Make It To Heaven
 > 0.849 - 50 Cent: Don't Push Me
 > 0.848 - 50 Cent: My Toy Soldier
 > 0.848 - 50 Cent: Gunz Come Out
 > 0.847 - 50 Cent: If I Can't
 > 0.844 - 50 Cent: This Is 50
 > 0.844 - 50 Cent: Wanksta - From "8 Mile

In [40]:
get_closest("Taylor Swift", "Style", 50)

Finding track uri


ValueError: can only call `.item()` if the dataframe is of shape (1, 1), or if explicit row/col values are provided; frame has shape (2, 1)

# Ignore Below

In [19]:
playlists = [row.split(",") for row in playlist_tracks["GROUP_CONCAT(PT.track_uri)"]]

In [21]:
del playlist_tracks

In [24]:
from gensim.models.callbacks import CallbackAny2Vec

class MonitorCallback(CallbackAny2Vec):
    def on_epoch_end(self, model):
        print("Model loss:", model.get_latest_training_loss())  # print loss
        print(model.wv.most_similar("spotify:track:1vrd6UOGamcKNGnSHJQlSt"))

In [ ]:
import logging
logging.basicConfig(level=logging.INFO)
model = Word2Vec(sentences=playlists, vector_size=25, window=5, min_count=50, sg=1, callbacks=[MonitorCallback()])
model.save("word2vec.model")

In [ ]:
sql_query = '''
    SELECT 
        artist.artist_name,
        track.track_uri, 
        track.track_name
    FROM track
    JOIN album ON track.album_uri = album.album_uri
    JOIN artist ON album.artist_uri = artist.artist_uri
'''
track_df = pd.read_sql_query(sql_query, conn)
track_df

In [ ]:
with open(r"C:\Users\liamb\Documents\Spotify-Million-Playlist-Challenge\song_embeddings.pkl", 'rb') as f:
    embeddings_dict = pickle.load(f)

In [ ]:
track_uris, embeddings = zip(*embeddings_dict.items())
track_uris = np.array(track_uris)
embeddings_matrix = np.stack([value for value in embeddings])
global_average_embedding = np.mean(embeddings_matrix, axis=0)

In [ ]:
def get_closest(artist: str, track: str, n: int):
    print("Finding track uri")
    track_uri = track_df[(track_df["artist_name"] == artist) & (track_df["track_name"] == track)]["track_uri"].values[0]
    print(f"Track URI: {track_uri}")
    print("Getting track embedding")
    track_emb = embeddings_dict[track_uri]
    print("Calculating scores")
    scores = np.dot(embeddings_matrix, track_emb)
    print("Getting top n indices")
    top_n_indices = np.argpartition(scores, n)[:n]
    print("Getting top n track uris")
    top_n_track_uris = track_uris[top_n_indices].tolist()
    print("Getting top n track data using URIs")
    return track_df[track_df["track_uri"].isin(top_n_track_uris)]

In [ ]:
get_closest("Taylor Swift", "Love Story", 50)

In [ ]:
query = '''
SELECT PT.pid, PT.track_uri, PT.pos
FROM playlist_track PT
INNER JOIN (
SELECT track_uri
FROM playlist_track
GROUP BY track_uri
HAVING COUNT(pid) > 1000
) AS PT2 ON PT.track_uri = PT2.track_uri
WHERE pid < 1000000
'''
playlist_tracks = pd.read_sql_query(query, conn)

In [ ]:
playlists = playlist_tracks.groupby('pid')['track_uri'].apply(list).tolist()

In [ ]:
model = Word2Vec(sentences=playlists, vector_size=100, window=5, min_count=1, sg=1)
track_embeddings = {track_uri: model.wv[track_uri] for track_uri in playlist_tracks['track_uri'].unique()}

In [ ]:
track_uris, embeddings = zip(*track_embeddings.items())
track_uris = np.array(track_uris)
embeddings_matrix = np.stack([value for value in embeddings])
global_average_embedding = np.mean(embeddings_matrix, axis=0)

In [ ]:
def get_closest(artist: str, track: str, n: int):
    print("Finding track uri")
    track_uri = track_df[(track_df["artist_name"] == artist) & (track_df["track_name"] == track)]["track_uri"].values[0]
    print(f"Track URI: {track_uri}")
    print("Getting track embedding")
    track_emb = track_embeddings[track_uri]
    print("Calculating scores")
    scores = np.dot(embeddings_matrix, track_emb)
    print("Getting top n indices")
    top_n_indices = np.argpartition(scores, n)[:n]
    print("Getting top n track uris")
    top_n_track_uris = track_uris[top_n_indices].tolist()
    print("Getting top n track data using URIs")
    return track_df[track_df["track_uri"].isin(top_n_track_uris)]

get_closest("Taylor Swift", "Love Story", 50)

In [ ]:
sql_query = '''
    SELECT 
        playlist.pid, 
        playlist.playlist_name,
        playlist.num_tracks 
    FROM playlist
'''
df = pd.read_sql_query(sql_query, conn)
df

In [ ]:
nlp = spacy.load('en_core_web_lg')

def word2vec(word: str):
    return nlp(word).vector

In [ ]:
# convert the name of the playlist into an embedding
def name_to_vec(name: str):
    return np.mean([word2vec(word) for word in name.split()], axis=0)

In [ ]:
df = df[df["num_tracks"] > 5]
len(df)

In [ ]:
df = df.sample(10_000)

In [ ]:
df["emb"] = df["playlist_name"].progress_apply(name_to_vec)

In [ ]:
df

In [ ]:
embeddings = df["playlist_name"].progress_apply(name_to_vec)

In [ ]:
embeddings.numpy()

In [ ]:
df.iloc[100]